In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('Tweets.csv');

In [5]:
df_final = df[['text','airline_sentiment']]

In [6]:
df_final

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
...,...,...
14635,@AmericanAir thank you we got on a different f...,positive
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative
14637,@AmericanAir Please bring American Airlines to...,neutral
14638,"@AmericanAir you have my money, you change my ...",negative


In [7]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'df' is your DataFrame and 'text_column' is the column name
label_encoder = LabelEncoder()
df_final['airline_sentiment'] = label_encoder.fit_transform(df_final['airline_sentiment'])


C:\Users\harsh\AppData\Local\Temp\ipykernel_13112\3571484346.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['airline_sentiment'] = label_encoder.fit_transform(df_final['airline_sentiment'])


In [8]:
df = df_final
df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,1
1,@VirginAmerica plus you've added commercials t...,2
2,@VirginAmerica I didn't today... Must mean I n...,1
3,@VirginAmerica it's really aggressive to blast...,0
4,@VirginAmerica and it's a really big bad thing...,0
...,...,...
14635,@AmericanAir thank you we got on a different f...,2
14636,@AmericanAir leaving over 20 minutes Late Flig...,0
14637,@AmericanAir Please bring American Airlines to...,1
14638,"@AmericanAir you have my money, you change my ...",0


In [9]:
df['text'] = df['text'].str.lower()

In [10]:
import string
df['text'] = df['text'].str.replace(r'[^\w\s]', '', regex=True)

In [11]:
df['text'] = df['text'].str.replace(r'\d+', '', regex=True)

In [12]:
df['text'] = df['text'].str.replace(r'[^a-zA-Z\s]', '', regex=True)


In [13]:
import contractions
df['text'] = df['text'].apply(lambda x:contractions.fix(x))

In [14]:
from nltk.tokenize import word_tokenize
df['text'] = df['text'].apply(word_tokenize)


In [15]:
df

,text,airline_sentiment
0,"[virginamerica, what, dhepburn, said]",1
1,"[virginamerica, plus, you, have, added, commer...",2
2,"[virginamerica, i, did, not, today, must, mean...",1
3,"[virginamerica, its, really, aggressive, to, b...",0
4,"[virginamerica, and, its, a, really, big, bad,...",0
...,...,...
14635,"[americanair, thank, you, we, got, on, a, diff...",2
14636,"[americanair, leaving, over, minutes, late, fl...",0
14637,"[americanair, please, bring, american, airline...",1
14638,"[americanair, you, have, my, money, you, chang...",0


In [19]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer ,WordNetLemmatizer
lemmetizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
# df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])
# Apply lemmatization and join the list back into a string
df['text'] = df['text'].apply(lambda x: ' '.join([lemmetizer.lemmatize(word) for word in x]))


In [20]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))



In [21]:
df

,text,airline_sentiment
0,virginamerica what dhepburn said,1
1,virginamerica plus you have added commercial t...,2
2,virginamerica i did not today must mean i need...,1
3,virginamerica it really aggressive to blast ob...,0
4,virginamerica and it a really big bad thing ab...,0
...,...,...
14635,americanair thank you we got on a different fl...,2
14636,americanair leaving over minute late flight no...,0
14637,americanair please bring american airline to b...,1
14638,americanair you have my money you change my fl...,0


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# # Fit and transform the reviews into a TF-IDF matrix
x = vectorizer.fit_transform(df['text'])
y = df['airline_sentiment']

In [23]:
y

0        1
1        2
2        1
3        0
4        0
        ..
14635    2
14636    0
14637    1
14638    0
14639    1
Name: airline_sentiment, Length: 14640, dtype: int32

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier
# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(x_train, y_train)

In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Number of trees
    'max_depth': [None, 10, 20, 30, 40, 50],   # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],           # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],             # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt'],          # Number of features to consider when looking for the best split
    'bootstrap': [True, False]                 # Whether bootstrap samples are used when building trees
}
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
rf_model = grid_search.best_estimator_

# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
# grid_search.fit(X_train, y_train)
# best_model = grid_search.best_estimator_


Fitting 3 folds for each of 1080 candidates, totalling 3240 fits


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
y_pred = rf_model.predict(x_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

0.7650273224043715